In [1]:
# importing the libraries
import pandas as pd
import numpy as np
import math
# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
# for creating validation set
from sklearn.model_selection import train_test_split
# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm
# PyTorch libraries and modules
import torch
import torch.nn as nn
import torchvision
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout


from torch.optim import Adam, SGD
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

import time

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay


# self write tools
from tools import LABEL_ENCODER, transforms_train, transforms_test
from models import CNN, RNN
from dataset import CnnDataset, TestDataset, CREATE_CNN_TRAIN_ITERATOR, CREATE_CNN_VALID_ITERATOR, words, TEST_DATASET







<IPython.core.display.Javascript object>

      id  category  gender  baseColour  season  usage  \
0  47381        21       4           2       3      0   
1  13218        24       2          45       2      2   
2  41776        24       4          31       0      0   
3  56271        12       2          25       2      0   
4  27361         9       2          32       1      0   

                              noisyTextDescription  \
0                              HM Women Blue Flats   
1                     Arrow Men Yellow Danny Shirt   
2  Chhota Bheem Kids Qsm I Love to red Pink Tshirt   
3           Jockey Textura Men Pack Of 2 Duet 1010   
4                         Wild stone Men Juice Deo   

                                       description  description_length  \
0                              HM Women Blue Flats                   4   
1                     Arrow Men Yellow Danny Shirt                   5   
2  Chhota Bheem Kids Qsm I Love to red Pink Tshirt                  10   
3           Jockey Textura Men Pack 

# Check GPU

In [2]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
torch.backends.cudnn.benchmark = True
print("Using {}: {}".format(device, torch.cuda.get_device_name(0)))

Using cuda: GeForce RTX 3080


# Global Variables

In [3]:

BATCH_SIZE = 256
EPOCHS = 40





In [4]:
class MyEnsemble(nn.Module):
    def __init__(self, _cnn, _rnn):
        super(MyEnsemble, self).__init__()
        self.cnn = _cnn
        self.rnn = _rnn
        self.classifier2 = nn.Linear(54, 54)
        self.classifier1 = nn.Linear(54, 54)
        self.classifier = nn.Linear(54, 27)
        
    def forward(self, image, description, l):
        x1 = self.cnn(image, description, l)
        x2 = self.rnn(description, l)

        x = torch.cat((x1, x2), dim=1)
        x = self.classifier2(F.relu(x))
        x = self.classifier1(F.relu(x))
        x = self.classifier(F.relu(x))
        return x

# 2. Model

In [5]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (image, (description, l), features, y) in iterator:

        description = description.to(device, dtype=torch.long)
        # l = l.to(device, dtype=torch.long)
        l = l.long()
        
        image = image.to(device)
        y = y.to(device, dtype=torch.int64)
        
        optimizer.zero_grad()
                
        # y_pred = model(image)
        y_pred = model(image)
        
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate_cnn(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():

        for (image, (description, l), features, y) in iterator:

            description = description.to(device, dtype=torch.long)

            l = l.long()
            
            image = image.to(device)
        
            y = y.to(device, dtype=torch.int64)
                    
            y_pred = model(image)
            
            loss = criterion(y_pred, y)
            
            acc = calculate_accuracy(y_pred, y)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)



In [6]:
cnn_train_iterator = CREATE_CNN_TRAIN_ITERATOR(transforms_train, BATCH_SIZE)
cnn_valid_iterator = CREATE_CNN_VALID_ITERATOR(BATCH_SIZE)

In [7]:
criterion = CrossEntropyLoss()
criterion = criterion.cuda()



cnn_model = CNN()
cnn_model = cnn_model.cuda()


model = cnn_model

optimizer = Adam(model.parameters(), lr=0.0001)





In [8]:




train_accuracy_list = []
train_loss_list = []
valid_acc_list = []
valid_loss_list = []
best_valid_loss = float('inf')

best_epoch = 0

for epoch in range(EPOCHS):

    start_time = time.monotonic()

    # train
    train_loss, train_acc = train(model, cnn_train_iterator, optimizer, criterion, device)
    
    # valid
    valid_loss, valid_acc = evaluate_cnn(model, cnn_valid_iterator, criterion, device)

    # save best model
    if valid_loss < best_valid_loss:
        best_epoch = epoch
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'output/cnn-model.pt')
        

    # Track the accuracy
    train_accuracy_list.append(train_acc)
    train_loss_list.append(train_loss)
    valid_acc_list.append(valid_acc)
    valid_loss_list.append(valid_loss)
        
    # print epoch info
    end_time = time.monotonic()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    
    if train_acc*100 > 99:
        break




Epoch: 01 | Epoch Time: 0m 36s
	Train Loss: 1.008 | Train Acc: 75.42%
	 Val. Loss: 0.402 |  Val. Acc: 89.30%
Epoch: 02 | Epoch Time: 0m 31s
	Train Loss: 0.327 | Train Acc: 90.84%
	 Val. Loss: 0.290 |  Val. Acc: 92.14%
Epoch: 03 | Epoch Time: 0m 31s
	Train Loss: 0.222 | Train Acc: 93.50%
	 Val. Loss: 0.278 |  Val. Acc: 92.02%
Epoch: 04 | Epoch Time: 0m 31s
	Train Loss: 0.163 | Train Acc: 95.16%
	 Val. Loss: 0.282 |  Val. Acc: 92.07%
Epoch: 05 | Epoch Time: 0m 31s
	Train Loss: 0.130 | Train Acc: 96.00%
	 Val. Loss: 0.264 |  Val. Acc: 93.14%
Epoch: 06 | Epoch Time: 0m 31s
	Train Loss: 0.098 | Train Acc: 97.01%
	 Val. Loss: 0.296 |  Val. Acc: 92.41%
Epoch: 07 | Epoch Time: 0m 31s
	Train Loss: 0.086 | Train Acc: 97.32%
	 Val. Loss: 0.295 |  Val. Acc: 93.30%
Epoch: 08 | Epoch Time: 0m 31s
	Train Loss: 0.073 | Train Acc: 97.69%
	 Val. Loss: 0.304 |  Val. Acc: 92.85%
Epoch: 09 | Epoch Time: 0m 31s
	Train Loss: 0.064 | Train Acc: 97.93%
	 Val. Loss: 0.301 |  Val. Acc: 92.87%
Epoch: 10 | Epoch T

In [9]:
print('Best Epoch:', best_epoch)


Best Epoch: 4


In [10]:
model.load_state_dict(torch.load('output/cnn-model.pt'))
test_loss, test_acc = evaluate_cnn(model, cnn_valid_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.264 | Test Acc: 93.14%


In [11]:

def predict(model):
    dataloader = DataLoader(TEST_DATASET, batch_size=1024)

    df = pd.DataFrame()
    model.eval()
        
    with torch.no_grad():

        for (images, (description, l), ids) in tqdm(dataloader):

            description = description.to(device, dtype=torch.long)
                # l = l.to(device, dtype=torch.long)
            l = l.long()
                
            images = images.to(device)
            
                        
            y_pred = model(images, description, l)
            

            top_pred = y_pred.argmax(1).cpu()
            labels = LABEL_ENCODER.inverse_transform(top_pred)

            temp = pd.DataFrame()
            temp['id'] = ids
            temp['category'] = labels
            

            df = df.append(temp)


    print(df)
    df.to_csv('output/cnn_prediction.csv', index=False)
